# DATA PREPARATION

In [3]:
import cv2
import shutil as s
import pandas as pd
from PIL import Image
from tqdm import tqdm
from os import listdir
from pdf2image import convert_from_path

In [4]:
CSV_PATH = "E:/7. SEMESTER 7/Bismillah Skripsi/Dataset/CSV Data/ECG_10001_Telemedicine_Analysed.csv"
PDF_PATH = "E:/7. SEMESTER 7/Bismillah Skripsi/Dataset/All Data"
PDF_FILES = listdir(PDF_PATH)
PDF_FILES.__len__()

10060

Retrieves the ID of the whole data from the .csv file

In [6]:
df = pd.read_csv(CSV_PATH)
df = df.drop_duplicates(subset=['ID'], keep='first', ignore_index=True)
ID = df['ID']
len(ID)

9540

Checking the ID that has the .pdf file

In [7]:
# Cek Not Avalailable Data
not_available_pdf_data = []

for i in ID:
    if i+'.pdf' not in PDF_FILES:
        not_available_pdf_data.append(i)
        

# Drop not Availabel Data
for i in not_available_pdf_data:
    indexData = df[df['ID'] == i].index
    df = df.drop(index=indexData)
    
# Reset Index
df = df.reset_index(drop=True)

# save csv data
df.to_csv('E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CSV FIX/data_bersih.csv')

All IDs from data_bersih.csv are copied from ALL DATA folder to ALL_PDF_FIX folder

In [8]:
CSV_FIX_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CSV FIX/data_bersih.csv'
ORIGINAL = 'E:/7. SEMESTER 7/Bismillah Skripsi/Dataset/All Data/'
TARGET = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/ALL_PDF_FIX/'

In [9]:
df_fix = pd.read_csv(CSV_FIX_PATH)

In [10]:
ID = df_fix['ID']

In [57]:
for i in tqdm(ID):
    s.copy(ORIGINAL + i + '.pdf', TARGET + i + '.pdf')

100%|██████████| 3604/3604 [01:56<00:00, 30.81it/s]


Converts .pdf files to .jpeg files

In [65]:
PDF_FILES = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/ALL_PDF_FIX/'
RESULT_DIR = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/ALL_IMG_FIX/'
FILE = listdir(PDF_FILES)

for f in tqdm(FILE):
    pdf_file = PDF_FILES + f

    pages = convert_from_path(pdf_file, poppler_path='C:/Program Files/poppler-0.68.0/bin')

    img_file = f.replace(".pdf", "")
    
    for page in pages[:1]:
        jpeg_file = RESULT_DIR + img_file + ".jpeg"
        page.save(jpeg_file, 'JPEG')

100%|██████████| 3604/3604 [45:40<00:00,  1.32it/s]  


Separate .pdf data with white and pink images. The data used is only pink data

In [70]:
df = pd.read_csv('E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CSV FIX/data_bersih.csv')
not_available_jpeg_data = listdir('E:/7. SEMESTER 7/Bismillah Skripsi/Dataset/Ping Putih Data/Putih/')

not_available_jpeg_data = [i[:-5] for i in not_available_jpeg_data]

# # Drop not Availabel Data
for i in not_available_jpeg_data:
    indexData = df[df['ID'] == i].index
    df = df.drop(index=indexData)
    
# # Reset Index
df = df.reset_index(drop=True)

df.to_csv('E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CSV FIX/CSV_PING.csv')

# IMAGE PROCESSING

Clean up unneeded data on the sides of the image by cropping the edges of the image

In [4]:
PING_IMG_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/ALL_PING_IMG/'
CROPPED_PING_IMG = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CROPPED_PING_IMG/'
ping_img_file = listdir(PING_IMG_PATH)

In [5]:
for i in tqdm(ping_img_file):
    im = Image.open(PING_IMG_PATH + i)
    
    left = 250
    top = 572
    right = 2216
    bottom = 1514

    im1 = im.crop((left, top, right, bottom))
    im1.save(CROPPED_PING_IMG + i)

100%|██████████| 2180/2180 [02:10<00:00, 16.70it/s]


Cleaning the image background to leave the image curve from noise

In [5]:
IMG_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CROPPED_PING_IMG/'
AMP_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/AMPLITUDE_IMG_FIX/'
img_files = listdir(IMG_PATH)

In [6]:
def denoising(img):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i][j] >= 100:
                img[i][j] = 255
            else:
                img[i][j] = 0

In [7]:
for i in tqdm(img_files):
    f = IMG_PATH + i
    img = cv2.imread(f, 0)
    denoising(img)
    cv2.imwrite(AMP_PATH + i, img)

100%|██████████| 2180/2180 [2:21:20<00:00,  3.89s/it]  


Cuts the image into 12 parts namely aVF, aVL, aVR, I, II, III, V1, V2, V3, V4, V5 and V6.

In [9]:
AMP_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/AMPLITUDE_IMG_FIX/'
CROPPED_12_PATH = 'E:/7. SEMESTER 7/Bismillah Skripsi/__BISMILLAH FIX__/CROPPED_12_PING_IMG/'
CROPPED_12_FILE = listdir(AMP_PATH)

In [25]:
point = [
    (0, 0, 489, 200),
    (0, 200, 489, 475),
    (0, 475, 489, 713),
    (491, 0, 982, 200),
    (491, 237, 982, 475),
    (491, 475, 982, 713),
    (983, 0, 1474, 200),
    (983, 200, 1474, 475),
    (983, 475, 1474, 713),
    (1475, 1, 1966, 200),
    (1475, 238, 1966, 475),
    (1475, 475, 1966, 713)
]

In [27]:
for i in tqdm(CROPPED_12_FILE):
    
    im = Image.open(AMP_PATH + i)
    width, height = im.size

    im0 = im.crop(point[0])
    im1 = im.crop(point[1])
    im2 = im.crop(point[2])
    im3 = im.crop(point[3])
    im4 = im.crop(point[4])
    im5 = im.crop(point[5])
    im6 = im.crop(point[6])
    im7 = im.crop(point[7])
    im8 = im.crop(point[8])
    im9 = im.crop(point[9])
    im10 = im.crop(point[10])
    im11 = im.crop(point[11])

    im0.save(CROPPED_12_PATH + 'I/' + i)
    im1.save(CROPPED_12_PATH + 'II/' + i)
    im2.save(CROPPED_12_PATH + 'III/' + i)
    im3.save(CROPPED_12_PATH + 'aVR/' + i)
    im4.save(CROPPED_12_PATH + 'aVL/' + i)
    im5.save(CROPPED_12_PATH + 'aVF/' + i)
    im6.save(CROPPED_12_PATH + 'V1/' + i)
    im7.save(CROPPED_12_PATH + 'V2/' + i)
    im8.save(CROPPED_12_PATH + 'V3/' + i)
    im9.save(CROPPED_12_PATH + 'V4/' + i)
    im10.save(CROPPED_12_PATH + 'V5/' + i)
    im11.save(CROPPED_12_PATH + 'V6/' + i)

100%|██████████| 2180/2180 [02:44<00:00, 13.22it/s]
